In [3]:
import numpy as np
import pandas as pd


In [4]:
df=pd.read_csv('../data/profit_full.csv')
df

,gold,estate,cryto,stock,class,risk,profit
0,14.96,41.76,1024.07,73.17,B,Y,463.26
1,25.18,62.96,1020.04,59.08,B,N,444.37
2,5.11,39.40,1012.16,92.14,B,N,488.56
3,20.86,57.32,1010.24,76.64,B,Y,446.48
4,10.82,37.50,1009.23,96.62,C,N,473.90
...,...,...,...,...,...,...,...
9563,16.65,49.69,1014.01,91.00,C,N,460.03
9564,13.19,39.18,1023.67,66.78,A,Y,469.62
9565,31.32,74.33,1012.92,36.48,A,N,429.57
9566,24.48,69.45,1013.86,62.39,A,N,435.74


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gold    9568 non-null   float64
 1   estate  9568 non-null   float64
 2   cryto   9568 non-null   float64
 3   stock   9568 non-null   float64
 4   class   9568 non-null   object 
 5   risk    9568 non-null   object 
 6   profit  9568 non-null   float64
dtypes: float64(5), object(2)
memory usage: 523.4+ KB


In [7]:
print(df.describe(include='all'))

               gold       estate        cryto        stock class  risk  \
count   9568.000000  9568.000000  9568.000000  9568.000000  9568  9568   
unique          NaN          NaN          NaN          NaN     3     2   
top             NaN          NaN          NaN          NaN     C     N   
freq            NaN          NaN          NaN          NaN  4433  4851   
mean      19.651231    54.305804  1013.259078    73.308978   NaN   NaN   
std        7.452473    12.707893     5.938784    14.600269   NaN   NaN   
min        1.810000    25.360000   992.890000    25.560000   NaN   NaN   
25%       13.510000    41.740000  1009.100000    63.327500   NaN   NaN   
50%       20.345000    52.080000  1012.940000    74.975000   NaN   NaN   
75%       25.720000    66.540000  1017.260000    84.830000   NaN   NaN   
max       37.110000    81.560000  1033.300000   100.160000   NaN   NaN   

             profit  
count   9568.000000  
unique          NaN  
top             NaN  
freq            NaN  
m

In [8]:
# biến mục tiêu
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import Normalizer,StandardScaler, MinMaxScaler,PowerTransformer,PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
column_trans=make_column_transformer(
    (OneHotEncoder(), ['class','risk']),
    (Normalizer(), ['cryto','stock']),
    remainder= 'passthrough'
)
pipe=Pipeline([
    ('columntransformer', column_trans),
    ('std_scaler', StandardScaler()),
    ('poly_fe', PolynomialFeatures(degree=4)),
    ('power_fe', PowerTransformer(copy=True,method='yeo-johnson',standardize=True)),
    ('mm_scaler', MinMaxScaler()),
    ('mlr_model', LinearRegression())
])


In [19]:
pipe.fit(X_train, y_train)
print('Training set score:', str(pipe.score(X_train, y_train)))
print('Test set score:', str(pipe.score(X_test, y_test)))

Training set score: 0.9453248520901422
Test set score: 0.9210177517080445


In [28]:
print(pipe.get_params().keys())

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'columntransformer', 'std_scaler', 'poly_fe', 'power_fe', 'mm_scaler', 'mlr_model', 'columntransformer__force_int_remainder_cols', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__onehotencoder', 'columntransformer__normalizer', 'columntransformer__onehotencoder__categories', 'columntransformer__onehotencoder__drop', 'columntransformer__onehotencoder__dtype', 'columntransformer__onehotencoder__feature_name_combiner', 'columntransformer__onehotencoder__handle_unknown', 'columntransformer__onehotencoder__max_categories', 'columntransformer__onehotencoder__min_frequency', 'columntransformer__onehotencoder__sparse_output', 'columntransformer__normalizer__copy', 'columntransformer__normalizer__norm', 'std_scal

In [29]:
print(pipe.get_params())

{'memory': None, 'steps': [('columntransformer', ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['class', 'risk']),
                                ('normalizer', Normalizer(),
                                 ['cryto', 'stock'])])), ('std_scaler', StandardScaler()), ('poly_fe', PolynomialFeatures(degree=4)), ('power_fe', PowerTransformer()), ('mm_scaler', MinMaxScaler()), ('mlr_model', LinearRegression())], 'transform_input': None, 'verbose': False, 'columntransformer': ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['class', 'risk']),
                                ('normalizer', Normalizer(),
                                 ['cryto', 'stock'])]), 'std_scaler': StandardScaler(), 'poly_fe': PolynomialFeatures(degree=4), 'power_fe': PowerTransformer(), 'mm_scaler': MinMaxScaler(), 'mlr_model': L

In [30]:
print(pipe.named_steps['mlr_model'].intercept_)
print(pipe.named_steps['mlr_model'].coef_)

-5067689828.868287
[-1.48e+04  1.46e+05  2.35e+05 -6.57e+05  3.70e+05  2.28e+05  2.76e+09
  2.12e+09 -1.87e+06 -8.02e+06  2.10e+05 -3.24e+05  2.38e+05  5.63e+06
 -5.62e+06  1.86e+08 -1.11e+09 -5.44e+07 -4.55e+05  3.34e+05  2.41e+05
 -2.88e+06  2.91e+06  1.64e+09  3.25e+09 -7.26e+07 -2.21e+07 -6.70e+05
 -4.13e+06  4.21e+06  2.83e+09  6.88e+08 -9.27e+04  6.70e+06  4.90e+05
 -4.87e+05  5.62e+08  5.99e+08  1.34e+06  2.06e+07  4.81e+05 -5.62e+08
 -5.99e+08 -1.34e+06 -2.15e+07  8.87e+05  2.24e+06  6.23e+03 -2.77e+04
  8.24e+05  4.04e+03 -1.78e+04  4.96e+02  6.12e+03  2.16e+03  2.63e+05
  3.28e+04 -7.29e+05  3.73e+06 -3.67e+06  9.34e+08 -2.48e+09  6.43e+07
 -5.32e+06 -1.29e+05  6.35e+05 -3.16e+06  3.17e+06  1.85e+08 -1.41e+09
 -7.72e+06  2.63e+07  3.84e+05 -4.04e+06  4.01e+06 -5.21e+08  3.04e+09
 -1.03e+07  5.49e+05 -6.88e+04  1.90e+05  7.14e+08 -1.13e+09 -4.71e+05
 -3.97e+06 -8.23e+04 -7.14e+08  1.13e+09  4.69e+05  3.94e+06 -2.55e+06
 -6.26e+07  1.86e+03 -7.85e+02  3.11e+07  5.41e+03 -1.32e+

In [31]:
y_pred=pipe.predict(X_test)
np.set_printoptions(precision=2)
print(y_pred)

[453.78 437.93 434.82 ... 456.42 471.44 448.81]


In [ ]:
import numpy as np


from sklearn import metrics
y_test_pred=pipe.predict(X_test)

print('Score or R-squared:',pipe.score(X_test, y_test))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print(' Root mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('Mean Absolute Percentage Error:', metrics.mean_absolute_percentage_error(y_test, y_test_pred))

Score or R-squared: 0.9210177517080445
Mean Absolute Error: 3.3258665875048936
 Root mean Squared Error: 22.95865607690566
Mean Absolute Percentage Error: 0.0073323366249662784


In [ ]:

from sklearn import metrics
best_params={
    'mlr_model__fit_intercept': False,
    'mlr_model__copy_X': True,
    'mlr_model__n_jobs': 1,
    'mlr_model__positive':True
}
pipe.set_params(**best_params)
pipe.fit(X_train, y_train)
print('Score or R-squared after tuning:', pipe.score(X_test, y_test))
y_test_pred=pipe.predict(X_test)
print('Score or R-squared after tuning:', pipe.score(X_test, y_test))
print('Mean Absolute Error', metrics.mean_absolute_error(y_test, y_test_pred))
print('Root Mean Squared Error', metrics.mean_squared_error(y_test, y_test_pred))
print ('Mean Absolute Percentage Error', metrics.mean_absolute_percentage_error(y_test, y_test_pred))

Score or R-squared after tuning: 0.9372937830334926
Score or R-squared after tuning: 0.9372937830334926


NameError: name 'metrics' is not defined

In [21]:
from sklearn.model_selection import RandomizedSearchCV
param_space = {
    'mlr_model__fit_intercept': [True, False],
    
    'mlr_model__copy_X': [True, False],
    'mlr_model__n_jobs': [1, 5],
    'mlr_model__positive': [True, False],
   
}
random_search = RandomizedSearchCV(pipe, param_space, n_iter=2,cv=5)
random_search.fit(X_train, y_train)
print('Best parameters found:', random_search.best_params_)
print('Best score:', random_search.best_score_)

KeyboardInterrupt: 

In [25]:
# xây dựng ứng dụng dự án profit
# gold =10.82 estate=37.50 scryto=1009.23 stock=96.62 class=A risk=Y
inputdf = pd.DataFrame([[10.82, 37.50, 1009.23, 96.62, 'A', 'Y']],
                       columns=['gold', 'estate', 'cryto', 'stock', 'class', 'risk'])
predicted_value = pipe.predict(inputdf)
print('Dự đoán lợi nhuận:', predicted_value)

Dự đoán lợi nhuận: [473.47355656]
